# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [1]:
from applets.helpers import get_connection, AppletBase

In [2]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [3]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [4]:
from applets.teapot import Teapot

# embit library: https://github.com/diybitcoinhardware/embit
# works with python3 and micropython
from embit.ec import PrivateKey, PublicKey, secp256k1, Signature

import os, hashlib, hmac
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

class MemoryCard(Teapot):
    def __init__(self, connection=None):
        super().__init__(connection)
        self.AID = "B00B5111CB01"
        self.iv = 0
        self.card_pubkey = None
        self.card_key = None
        self.host_key = None
        self.mode = "es"

    def get_random(self):
        return self.request("B0B10000")
    
    def get_card_pubkey(self):
        sec = self.request("B0B2000000")
        self.card_pubkey = PublicKey.parse(sec)
        return self.card_pubkey

    def establish_secure_channel(self, mode=None):
        # save mode for later - i.e. reestablish secure channel
        if mode is None:
            mode = self.mode
        else:
            self.mode = mode
        # check if we know pubkey already
        if self.card_pubkey is None:
            self.get_card_pubkey()
        # generate ephimerial key
        secret = os.urandom(32)
        host_prv = PrivateKey(secret)
        host_pub = host_prv.get_public_key()
        host_pub.compressed = False
        data = bytes([65])+host_pub.sec()
        # ee mode - ask card to create ephimerial key and send it to us
        if mode=="ee":
            # get ephimerial pubkey from the card
            res = app.request("B0B40000"+data.hex())
            pub = PublicKey.parse(res[:65])
            secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
            shared_secret = pub.sec()[1:33]
            self.host_key = hashlib.sha256(b'host'+shared_secret).digest()
            self.card_key = hashlib.sha256(b'card'+shared_secret).digest()
            shared_hash = hashlib.sha256(self.host_key+self.card_key).digest()
            recv_hmac = res[65:97]
            h = hmac.new(self.card_key, digestmod='sha256')
            h.update(res[:65])
            expected_hmac = h.digest()
            if expected_hmac != recv_hmac:
                raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
            sig = Signature.parse(res[97:])
            # card doesn't follow low s rule, so we need to normalize
            sig._sig = secp256k1.ecdsa_signature_normalize(sig._sig)
            if not self.card_pubkey.verify(sig, hashlib.sha256(res[:97]).digest()):
                raise RuntimeError("Signature is invalid: %r", res[97:].hex())
        # se mode - use our ephimerial key with card's static key
        else:
            pub = PublicKey.parse(self.card_pubkey.sec())
            secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
            shared_secret = pub.sec()[1:33]
            self.host_key = hashlib.sha256(b'host'+shared_secret).digest()
            self.card_key = hashlib.sha256(b'card'+shared_secret).digest()
            shared_hash = hashlib.sha256(self.host_key+self.card_key).digest()
            res = self.request("B0B30000"+data.hex())
            recv_hmac = res[32:64]
            h = hmac.new(self.card_key, digestmod='sha256')
            h.update(res[:32])
            expected_hmac = h.digest()
            if expected_hmac != recv_hmac:
                raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
            sig = Signature.parse(res[64:])
            # card doesn't follow low s rule, so we need to normalize
            sig._sig = secp256k1.ecdsa_signature_normalize(sig._sig)
            if not self.card_pubkey.verify(sig, hashlib.sha256(res[:64]).digest()):
                raise RuntimeError("Signature is invalid")
            if res[:32] != shared_hash:
                raise RuntimeError("Meh... Something didn't work? Card returned %s, we have %s" % (res[:32].hex(), shared_hash.hex()))
        # reset iv
        self.iv = 0
    
    def encrypt(self, data):
        # add padding
        d = data+b'\x80'
        if len(d)%16 != 0:
            d += b'\x00'*(16 - (len(d)%16))
        iv = self.iv.to_bytes(16, 'big')
        backend = default_backend()
        cipher = Cipher(algorithms.AES(self.host_key), modes.CBC(iv), backend=backend)
        encryptor = cipher.encryptor()
        ct = encryptor.update(d)+encryptor.finalize()
        h = hmac.new(self.host_key, digestmod='sha256')
        h.update(iv)
        h.update(ct)
        ct += h.digest()
        return ct
    
    def decrypt(self, ct):
        recv_hmac = ct[-32:]
        ct = ct[:-32]
        iv = self.iv.to_bytes(16, 'big')
        h = hmac.new(self.card_key, digestmod='sha256')
        h.update(iv)
        h.update(ct)
        expected_hmac = h.digest()
        if expected_hmac != recv_hmac:
            raise RuntimeError("Wrong HMAC. Got %s, expected %s"%(recv_hmac.hex(),expected_hmac.hex()))
        backend = default_backend()
        cipher = Cipher(algorithms.AES(self.card_key), modes.CBC(iv), backend=backend)
        decryptor = cipher.decryptor()
        # check and remove \x80... padding
        plain = decryptor.update(ct)+decryptor.finalize()
        arr = plain.split(b"\x80")
        if len(arr)==1 or len(arr[-1].replace(b'\x00',b''))>0:
            raise RuntimeError("Wrong padding")
        return (b"".join(arr[:-1]))
    
    def secure_request(self, data):
        # if counter reached maximum - reestablish channel
        if self.iv >= 2**16:
            self.establish_secure_channel()
        ct = self.encrypt(data)
        res = app.request("B0B50000"+(bytes([len(ct)])+ct).hex())
        plaintext = self.decrypt(res)
        self.iv += 1
        return plaintext

In [5]:
app = MemoryCard(connection)
app.select()

b''

In [6]:
app.get_random().hex()

'8663107fb2b6ef01b727f76cdf879001ba9d47362f1b422977321e46caa660ea'

# 2. Establish secure communication

At the moment we use a homebrew version, I need to understand and implement Noise in the near future.

[Noise protocol](https://noiseprotocol.org/noise.html) should be with the following primitives:
- secp256k1 as signature scheme
- AES CBC with random IV as a symmetric cipher
- SHA256 as a hash function, HMAC_SHA256 as authentication code

AES CBC because we have hardware implementation on the card, so we can use the Key class that is more secure than byte array. HMAC sucks here though...

Generally a shared secret is split to 2 different secrets for encryption and decryption. This allows us to use AES in counter mode where the counter is the same for both host and the card - they use different keys so it's ok.

Description of the current protocol:
1. Get the static public key from the card. If it is already known - proceed with step 2
2. Generate a random key pair and send corresponding pubkey in uncompressed form to the card: `<data:pub_uncompressed>`
3. Card will return its fresh session public key, hmac of this public key using shared secret and signature: `<data:session_pubkey_card><hmac(dec,data)><sig(static_pubkey_card,data|hmac)>`
4. Verify the signature against card's pubkey
5. Calculate a pair of shared secrets from `enc=SHA256('host'|ECDH(d,session_pubkey_card))`, `dec=SHA256('card'|ECDH(d,session_pubkey_card)`
6. Verify that `hmac==HMAC_SHA256(dec,session_pubkey_card)`
7. Set counter to zero `iv=0`, use `dec` for decryption, `enc` for enctyption. Increase the counter on every request, reestablish communication if the counter reaches max value (16 bytes, shouldn't happen in most cases)
8. All secure messages now should be in the form `<ct:aes(enc,iv,command|data)><hmac_sha256(enc,iv|ct)>`, `iv` is not transmitted but increased for every new command and used in HMAC. Returned data will be of the same form but using the `dec` key. `iv` will be the same.

Maximum size of the APDU is 256 bytes. Maximum size of the mnemonic is `9*24-1=215` bytes plus length and command and rounding to `16` bytes, plus `32`-byte hmac - we are at `256` bytes... We can send at max `255` bytes.

So to store and retrieve large mnemonic we need to split it to two pieces, and provide a function to set part of the data array. Therefore for secret storage we have to make an API: 
- `set_data<data>` / `get_data` - for short mnemonics
- `set_data_length<len>` - should be called first, erases existing data, 
- `set_data_part <start><data>` - stores part of the data with an offset
- `get_data_length`
- `get_data_part <start><len>` - gets part of the data from `start` with `len`


In [7]:
app.establish_secure_channel() # mode="se" and mode="ee" available, "se" by default

In [8]:
app.secure_request(b'ping')

b'qjoh'

In [9]:
connection.disconnect()